# CH413 Computational Workshop 2: 
# Predicting the stability of an oxygen-covered diamond(110) surface

## Introduction

**ATTENTION** Your very first action should be to safe a copy of this notebook and to rename it

<p>In this workshop, we will be exploring Density Functional Theory (DFT) calculations and their application in surface chemistry. We will do this using the ASE and GPAW packages (Click on images to go to webpages):</p>

<div style="display: flex;">
  <div style="flex: 40%;padding: 15px;">
    <a href="https://wiki.fysik.dtu.dk/ase/"><img src="./DFT_MM_MAURER/ase.png" width="80%"></a>
  </div>
  <div style="flex: 40%;padding: 15px;">
     <a href="https://wiki.fysik.dtu.dk/gpaw/"><img src="./DFT_MM_MAURER/gpaw.png" width="100%"></a>
  </div>
</div>


<br> 

### ASE
<b>ASE</b> is a Python package that provides simple workflows for computational chemistry and materials modeling tasks and interfaces with many different DFT software packages. ASE uses Python computer language to define what atoms are. It defines representations for molecules and materials, allows us to create, visualise, and manipulate them.

For example, we can define an oxygen molecule in ASE and visualise it inside this Jupyter notebook:

In [ ]:
from ase import Atoms
from ase.visualize import view

#We define a so-called Atoms object ("collection of atoms") with the chemical formula o2
o2 = Atoms('O2', 
          positions = [
              [0,0,0],   #x,y,z positions of 1st oxygen atom
              [0,0,1.0], #x,y,z positions of 2nd oxygen atom at distance 1 Angstrom from 1st atom
          ],
          )

#visualise o2
#There are two different viewing modes viewer="ngl".
#If you don't put the viewer option, it doesn't work (yet).
view(o2, viewer='ngl')
#In the ngl option, please select the color scheme 'element'
#You can now interact with the viewer, rotate the molecule etc.

In [ ]:
###With the ngl viewer, we can manipulate the visualisation. Looks daunting, I know, but here are a few examples###
v=_ #pick up visualisation from previous cell
v.view.clear_representations()
v.view._remote_call("setSize", target="Widget", args=["400px", "400px"])
v.view.add_ball_and_stick(opacity=0.8) #draw oxygen as transparent ball and stick model 
v.view.background='#ffc' #change background color to light yellow
v.view.camera = 'perspective' #change camera perspective


We can use ASE to generate surface slab structures. Please take a look at this short ASE tutorial on <a href="https://wiki.fysik.dtu.dk/ase/tutorials/surface.html">Nitrogen on Cu(111) </a>

**TASK** After going through the tutorial, pick the relevant commands to generate the N$_2$ on Cu(111) surface and paste them into the following cell (don't forget the import commands) and visualise the structure (use the same command as above).

In [ ]:
##Use this code cell to visualise the N2 on Cu(100) system
from ase.build import fcc111, add_adsorbate

#####ADD CONTENT HERE#####

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())
view(slab,viewer="ngl")

#
#You should be able to see the surface, the molecule, and the edge of the unit cell

In [ ]:
#Lets make the visualisation nicer again
v=_
v.view.clear_representations()
v.view.add_ball_and_stick(opacity=0.8)
v.view.add_unitcell()
##Here's a few other interesting visualisation types
#v.view.add_hyperball()
#v.view.add_spacefill()

### GPAW

One of the software packages that can be interfaced with ASE is <b>GPAW</b> (an abbreviation for "Grid-based Projector-Augmented-Waves", which is the basis set that GPAW uses). It is a Python/C++-based DFT code. GPAW describes the Kohn-Sham DFT wave functions with a basis set that splits the core electrons and the valence electrons. Core electrons are "frozen" and described with projector-augmented waves (PAWs) - a formalism that is closely related to the pseudopotentials we discussed in the lecture. The valence electron states are described with a basis set. That basis set in GPAW is usually a uniform real-space grid. Each grid point is a "basis function". You can read more about this <a href="https://wiki.fysik.dtu.dk/gpaw/algorithms.html">here</a>.

GPAW defines a so-called "calculator" - a DFT engine that provides energies and forces to the "atoms" object. 
ASE and GPAW are linked by providing a GPAW calculator to the ASE atoms object.
<br>
Let's use GPAW to calculate the properties of the oxygen molecule:

In [ ]:
from ase import Atoms
from ase.io import write
from gpaw import GPAW

#Let's use the o2 molecule we set up before
print(o2)

**ATTENTION**
The pint function shows that the o2 molecule currently doesn't have any periodic boundary conditions defined (pbc=False). We can not define a grid or plane-wave basis without periodic boundary conditions, because we need Born-von-Karman conditions and an infinite lattice. That means we first need to put the oxygen molecule in a big box (a so-called supercell).

In [ ]:
#Let's take to o2 molecule and put it into a box
#We simply "give" the molecule a unit cell defined by three lattice vectors a1, a2, a3

a = 6.0
cell = [
    [a,0,0], #a1
    [0,a,0], #a2
    [0,0,a], #a3
]
#This is a cubic box with 6 Angstrom side length
o2.set_cell(cell)
#Now we set the periodic boundary conditions to True in all three directions and we center the molecule in the cell
o2.center()
o2.set_pbc([True,True,True])
print(o2)
view(o2,viewer='ngl')

In [ ]:
#Let's set up a GPAW calculator. Here we need to choose numerical settings as discussed in the lecture.
calc_o2 = GPAW(h=0.25, # basis set defined by a grid spacing in Angstrom
            xc="PBE", #choice of exchange-correlation functional
            kpts=(1,1,1), #k-grid sampling: here we only pick a single K point, 
            #because we want to describe oxygen as a single molecule
           )

#h value: The grid spacing defines the basis set. We chose a grid spacing of 0.25 Angstrom in all directions. 
#The smaller the number, the better the basis set is converged, 
#but also the more computationally costly the calculation becomes.

#Now we "give" the calculator to the atoms o2 molecule
o2.set_calculator(calc_o2)

#Now we calculate the total energy for o2 using the GPAW calculator with the chosen settings
etot_o2 = o2.get_potential_energy()
print("etot", etot_o2)
#When running this cell, you should see a pop-up of the output file, where you can follow the calculation.

**TASK**
Carefully study the output that was generated by this DFT calculation.
Study the output file and look for following things:
* A basic description of all settings and the geometry
* The actual DFT calculation, where the Kohn-Sham (KS) equations are solved iteratively with many self-consistent field (SCF) iteration steps
* The final total energy (in eV) and the energies and occupation numbers of the Kohn-Sham states

After this, we can now also optimise the bond distance of the O$_2$ molecule:

In [ ]:
#we import a geometry optimisation algorithm
from ase.optimize import BFGS
#we apply it to the o2 molecule object (which contains the GPAW calculator)
dyn = BFGS(o2,trajectory='o2_opt.traj')
#we run the optimisation, but before that, we pipe the GPAW text output into a file, so we're not overwhelmed by the output
calc_o2.set(txt="o2_gpaw.txt")
dyn.run(fmax=0.05)
#It might take a while before the steps appear...

In each step of the geometry optimisation, the algorithm calculates the energy and forces from DFT and changes the geometry towards lower energy. This is repeated until the maximum force acting on the atoms is reduced to the threshold value 0.05 eV/Angstrom. Let's check out the optimised bond distance:

In [ ]:
print('bond distance in Angstrom', o2.get_distance(0,1))
etot_o2 = o2.get_potential_energy() # let's update the new optimised o2 energy
print('etot_o2', etot_o2)

Alright, so much for the basics. Let's get into the meat of things.
This notebook has three parts:
* <b>Part 1</b>: Calculating the adsorption energy of oxygen on diamond(110) with DFT
* <b>Part 2</b>: Calculating stability phase diagrams for oxygen-covered diamond phases
* <b>Part 3</b>: Assessed part of the Workshop with work to do on your own


## Part 1: Calculating the adsorption energy of oxygen on diamond(110): 

Diamond, beyond being shiny and expensive in some of its variants, is an interesting material in chemistry and physics. Particularly for electrochemical applications, boron-doped diamond can be used as a versatile electrode material. Despite the decades of research work that exist on diamond and its materials and surface properties, many details are not full understood. For example the diamond(110) surface termination, a common termination for ion-milled diamond, is known to be oxygen-terminated, but the nature of this termination is not fully clear. 
The following picture, shows some examples of possible oxygen functional groups that can sit on the carbon ridges that are exposed at the diamond(110) surface:

<img src="./DFT_MM_MAURER/diamond.png" width="100%">
Reference: Mackey et al., J. Phys. Chem. B 2001, 105, 3803-3812

To resolve this, we need to model different possible surface terminations of diamond surfaces and study, which are the energetically most favorable ones. Let's start by taking a look at a clean diamond(110) surface.


In [ ]:
from ase.io import read
from ase.visualize import view

#The folder DFT_MM_MAURER/structures contains several different structure files
clean_diamond = read('DFT_MM_MAURER/structures/clean_bdd.in')
#the file contains a minimal surface unit cell of diamond(110) with 7 layers of carbon atoms
view(clean_diamond, viewer="ngl")

#We can visualise the surface better by repeating it periodically in x and y direction
#COMMENT OUT THE NEXT LINE
#view(clean_diamond*[5,5,1], viewer="ngl") #5 repeats in x and y direction

In [ ]:
#Lets make the visualisation nicer again
v=_
v.view.clear_representations()
v.view.add_ball_and_stick(opacity=0.8)
v.view.add_unitcell()

The adsorption energy per oxygen atom is defined as
$$ E_{\mathrm{ads}}=-\frac{1}{N_O}\left( E^{\mathrm{slab}}_{\mathrm{surf}}(N_O,N_M)-\left( E^{\mathrm{clean}}_{\mathrm{surf}}(0,N_M) + \frac{N_O}{2}E^{\mathrm{total}}_{O_2}  \right)  \right) $$

That means we need to calculate the energy of the isolated oxygen molecule, the clean diamond surface, and the oxygen-covered diamond surface. We already calculated the oxygen molecule, let's calculate the clean diamond surface energy.

In [ ]:
#Let's set up a new calculator for the clean surface
calc_clean = GPAW(h=0.25, #basis grid spacing
             xc="PBE", 
             kpts=(2,2,1), #a 2x2x1 Monkhorst-Pack grid. 2 k points in a1 and 2 k points in a2 direction
                  #we dont need to do k sampling perpendicular to the surface, as we want this to be 'non-periodic'
           )

clean_diamond.set_calculator(calc_clean)
etot_clean = clean_diamond.get_potential_energy()
print('etot_clean', etot_clean)
#dont be surprised if the calculation takes a little bit. You can follow the convergence in the output window

Alright, now let's check out one of the oxygen-covered phases shown in the above figure. Let's start with a keto group. You'll find different preoptimised diamonds tructures in the folder DFT_MM_MAURER/structures/:

In [ ]:
diamond_ketone = read('DFT_MM_MAURER/structures/phase1.in')
view(diamond_ketone*[5,5,1], viewer="ngl") #we are visualising the 5fold repeated cell

In [ ]:
#Lets make the visualisation nicer again
v=_
v.view.clear_representations()
v.view.add_ball_and_stick(opacity=0.8)
v.view.add_unitcell()

In [ ]:
#Let's set up a new calculator for the keto phase surface
calc = GPAW(h=0.25, 
             xc="PBE", 
             kpts=(2,2,1), 
           )

diamond_ketone.set_calculator(calc)
etot_keto = diamond_ketone.get_potential_energy()
print('etot_keto', etot_keto)

Now we can calculate the adsorption energy as defined by the equation above:

In [ ]:
e_ads = -(etot_keto -(etot_clean+etot_o2/2) )
print ("e_ads", e_ads)

The adsorption energy of this oxygen atom is about 1.41 eV per oxygen atom. We can also calculate this as adsorption energy per surface area by dividing by the surface area of the unit cell:

In [ ]:
#The surface area of the unit cell is defined as the absolute value of the cross product between the a1 and a2 lattice vectors
cell = diamond_ketone.get_cell()
print(cell)
a, b, c = cell
#let's use the linear algebra operations in numpy for this
import numpy as np
area = np.linalg.norm(np.cross(a,b))
print('area', area)
#In this simple rectangular unit cell case, it would also have been enough to multiply 2.52 with 3.57
print('area', a[0]*b[1])

#The adsorption energy per surface area and oxygen atom is:
print('e_ads per area', e_ads/area)

## Part 2: Calculating stability phase diagrams for different oxygen-covered diamond phases

Now that we have an adsorption energy, we can actually plot this as a function of chemical potential just like we discussed it in the lecture. Just as a reminder, here's the equation we use for that:

$$ Δ𝐺^{\mathrm{𝑎𝑑}} (Δ\mu_𝑂)≈\frac{𝑁_𝑂}{𝐴} 𝐸_{\mathrm{ads}}+(𝑁_𝑀−𝑁_𝑀^′ ) 𝐸_M^{bulk}+\frac{𝑁_𝑂}{𝐴} Δ\mu_𝑂 (𝑇,𝑝) $$

Let's plot this in a graph of $$ Δ𝐺^{\mathrm{𝑎𝑑}} vs. Δ\mu_𝑂 $$


We already have calculated the adsorption energy per surface area and oxygen atom:

In [ ]:
print('e_ads per area', e_ads/area, 'eV/Angstrom')
e_b = e_ads/area


The adsorption free energy is a straight line with the y-intercept defined by the adsorption energy per surface area and oxygen atom and the gradient defined by $N_O/A$:

In [ ]:
gradient = 1/area
print('gradient', gradient, '1/Angstrom')

Let's generate that phase diagram. Here, we need to use the above equation to generate adsorption free energy values at certain values of chemical potential:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook 
#this gives us an interactive graph with a cross hair

mu = np.arange(-2.50,0.1,0.1) # This generates mu values for the x-axis from -2.5 to 0 with a spacing of 0.01 eV

#let's generate the y value for our plot for a given set of mu values
#This is a very Pythonic ("integrated and neat") way to do that
Gads = [ e_b+x*gradient for x in mu ] #We loop over all mu values (caling them x) and calculate the G_ads value

################################################

plt.plot(mu,Gads, color='red',label='phase 1')

#we should also plot the line for the clean surface, which is a straight horizontal line at 0
plt.plot(mu, [0 for m in mu], color='black', label='clean surface')


#lets not forget that we want these plots to have inverted y axis, 
#because a bigger value means a more stable phase
plt.ylim(0.30, -0.10) 
plt.xlim(-2.0,0)
plt.xlabel(r'chemical potential $\Delta\mu$ in eV') 
#by the way, this is how you get latex math into the label, dont forget the r in front
plt.ylabel(r'adsorption free energy $\Delta G^{ads}$ in eV')
plt.legend()
plt.text(-1.90, 0.15,'clean surface') #add text to the graph
plt.text(-0.90, 0.15,'phase 1')

plt.fill([-4.0,-1.41, -1.41, -4.0],[0.0,0.0, 0.40, 0.40],'gray', alpha=0.3) # let's fill the surface area for the clean surface
plt.fill([-1.41,-1.41,0.0,0.0],[0.40,0.0, 0.15616,0.40],'b', alpha=0.3) #let's fill the surface area for phase 1 with a 

plt.axvline(-1.41,color='gray',ls=':') #This is a vertical line to mark the intersection between the two phases

We can see that the two curves cross at about -1.41 eV chemical potential. This is the point where the keto-terminated oxygenated diamond(110) surface becomes more stable than the clean diamond(110) surface.

Of course, we should also try to find out which other phases exist and how stable they are. This is going to be your job for the assessement in Part 3.


## Part 3: Assessed part of the Workshop

## TASK 1: Convergence testing


As discussed in the lecture, we should check the convergence of this adsorption energy by varying the basis set cutoff (the grid spacing h) and by varying the k-grid.

1. Recalculate the energy of oxygen, the clean surface, and the keto phase with different settings to study the convergence of the adsorption energy, first with basis set, and then with k-point.
    * Vary the basis grid spacing between [0.25,0.20,0.15,0.10], Note: stick with 2x2x1 k-grid for the surfaces and a 1x1x1 grid for o2
    * Vary the k-grid between [(1,1,1), (2,2,1), (3,3,1), (4,4,1)], Note: don't vary it for the o2 molecule, stick with 1x1x1
    
    If you think it is necessary, you should also explore points inbetween these settings
2. Generate two plots of
$E_{ads}$ vs. basis (0.25,0.20,0.15,0.12) and $E_{ads}$ vs. k point (1,2,3,4)

    Below you'll find some prepared functions to help you to plot the results. Pleae correctly label the axes.
3. Discuss the convergence behavior and discuss in detail if any of the chosen settings would be numerically precise enough to predict the adsorption energy of different oxygen-diamond phases up to a precision of $\pm$0.10 eV. (Note: Discuss the precision, not the method accuracy.)

*Attention* Some of the calculations with higher settings may run a few minutes and require a bit of patience


In [ ]:
#####calculations, use multiple cells####


basis = [0.25,0.20,0.15,0.10] # feel free to also explore points inbetween
Eads_basis = []

####FILL IN
#for b in basis:




####FILL IN



In [ ]:
####Basis Convergence Plotting#####
import numpy as np

from matplotlib import pyplot as plt
%matplotlib notebook 

#plt.ylim(low, high)
### ylabel


In [ ]:

ks = [1,2,3,4] # feel free to also explore points inbetween
Eads_ks = []
####FILL IN
#for k in ks:

    
####FILL IN
    

In [ ]:
####K point Convergence Plotting#####

####FILL IN
import numpy as np

from matplotlib import pyplot as plt
%matplotlib notebook 






**DISCUSSION OF RESULTS for TASK 1**


Enter discussion text below using Markdown and Latex syntax, where necessary. Google 'Markdown cheatsheet'

####PLEASE DOUBLE CLICK HERE AND ENTER TEXT USING LATEX and Markdown



## Task 2: Phase Diagram of various oxygen-covered diamond(110) surfaces

In addition to the structure discussed in Parts 1 and 2, there are many other possible oxygen terminations.

1. Visualise the additional oxygen terminated phases provided in DFT_MM_MAURER and calculate the surface area and number of oxygen-atoms-per-surface-area ratio for each.
   * phase2.in (keto phase with 2 oxygen atoms per unit cell)
   * phase4.in (5-ring ether phase)
   * phase5.in (peroxo phase)

2. Use the settings that you have identified in Task 1 to calculate the adsorption energies for all 4 phases (1,2,4,5)

3. Plot the phase diagram of adsorption free energy ($\Delta G^{ad}$) vs. change in chemical potential ($\Delta\mu_O$). 
   
   Note: In all cases, the number of carbon atoms is the same, so the contribution from the change in bulk atoms can be ignored
   
4. Identify the most stable phases for each region of chemical potential. Note and discuss this in a comment cell

5. Translate the phase diagram for a change in chemical potential to a Temperature vs. Pressure phase diagram in the temperature range of 100 K to 1200K and pressure range of 10E-9 atm to 1atm.

   Do this using following expression: 
   $$ \Delta\mu(T,p)= a\cdot T^4+b\cdot T^3+c\cdot T^2+d\cdot T+ e + \frac{1}{2}k_BT\ln{\frac{p}{p^0}}$$
   where $T^0=298.15$ K and $p^0=1$ atm and $a=-4.231\cdot 10^{-12}$ eV/K, $b=6.505\cdot 10^{-9}$ eV/K, $c=-3.401\cdot 10^{-6}$ eV/K, $d=-1.259\cdot 10^{-3}$ eV, $e=-8.883\cdot 10^{-2}$ eV
   
   Below I have written a function that provides you with the chemical potential, given certain values of T [in K] and p [in atm]

6. Discuss all your results in detail and answer following questions:
   * What is the most stable phase at ambient pressure and temperature conditions?
   * What happens when we heat the surface (assuming no kinetic barriers)?
   * What happens when we leave the temperature constant and increase the pressure?
   * In the oxygen on Pd(100) example, we also had a bulk oxide phase involved. What would be the 'bulk oxid' for diamond? Under which conditions would this bulk oxide become relevant?


In [ ]:
def deltamu_Tp(T,p):
    """
    takes T in Kelvin and p in atm and converts to Delta mu in eV
    The function is a polynomial fit based on the data provided here:
    https://janaf.nist.gov/tables/O-029.html
    """
    from math import log
    a = -4.231E-12
    b =  6.505E-09
    c = -3.401E-06
    d = -1.259E-03
    e = -8.883E-02
    kB = 8.6173E-05
    
    mu_Tp0 = a*(T**4)+b*(T**3)+c*(T**2)+d*T+e
    mu_Tp = mu_Tp0 + 0.5*T*kB*log(p) 
    return mu_Tp

#### Example: The value of chemical potential at 600K and standard pressure is 
print(deltamu_Tp(600,1))

In [ ]:
#####VISUALISATION and gradient



In [ ]:
#####Adsorption energy calculations

#o2


#clean

#phases



In [ ]:
# calculate adsorption energies



In [ ]:
####Phase diagram plot G vs. mu

import matplotlib.pyplot as plt
%matplotlib notebook 
#this gives us an interactive graph with a cross hair



##### List of most stable phases

mu range -X.0 - -Y.0 : Phase X is most stable
etc...



In [ ]:
#####Translate phase diagram into T vs. p diagram
#HINT: First, generate values of mu for a range of temperatures (100, 200, etc. steps of hundred) and 
#pressures (10E-9,10E-8, etc. logarithmic)

#Generate a two-dimensional numpy array of mu values mu = np.zeros[(N_temps,N_pressures)] and fill it with mu values.
#For example:
import numpy as np
import matplotlib.pyplot as plt
x, y = np.meshgrid([T1,T2,T3,...], [....])
z = deltamu_Tp(x,y)
##Now assign each value of mu with a number identifier for the phase (1,2,3,...)




#Check out following examples to learn how to plot a contour plot
#   https://matplotlib.org/examples/pylab_examples/contour_demo.html
#   https://matplotlib.org/gallery/images_contours_and_fields/contour_corner_mask.html
    




**DISCUSSION OF RESULTS for TASK 2**


Enter discussion text below using Markdown and Latex syntax, where necessary. Google 'Markdown cheatsheet'

####PLEASE DOUBLE CLICK HERE AND ENTER TEXT USING LATEX and Markdown


